## Домашнее задание по блоку 22.03.2021. Основы парсинга и работы с API
### Тихонова Инна

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re
import json

**Задание 1. Обязательная часть**

Будем парсить страницу со свежими новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [2]:
KEYWORDS = ['python', 'парсинг', 'Мурманск', 'статья из цикла', '2021', 'безопасно']

In [3]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'lxml')

In [4]:
posts = soup.find_all('article', class_='post post_preview')

In [5]:
def get_correct_datetime(date_text):
    
    #словарь для перевода строки с датой в формат даты
    dict_month = {'января': '01', 'февраля': '02', 'марта': '03'} #....
    #сегодняшняя дата, 2021-03-23
    today = datetime.now()
    
    #в поле "дата" у новости может быть не только дата, но и "сегодня" или "вчера". Перевела это в datetime
    if 'сегодня' in date_text:
        # 'сегодня в 10:47'
        date_str = date_text.replace('сегодня в', today.strftime("%d %m %Y"))

    elif 'вчера' in date_text:
        # 'вчера в 10:47'
        date_str = date_text.replace('вчера в', (today + timedelta(days=-1)).strftime("%d %m %Y"))

    else:
        # '21 марта 2021 в 10:00'
        for month in dict_month:
            date_text = date_text.replace(month, dict_month[month])
        date_str = date_text.replace(' в', '')

    date_correct = datetime.strptime(date_str, '%d %m %Y %H:%M')
    
    return date_correct

In [6]:
#тест
print(get_correct_datetime('сегодня в 10:47'))
print(get_correct_datetime('вчера в 10:47'))
print(get_correct_datetime('21 марта 2021 в 10:00'))

2021-03-23 10:47:00
2021-03-22 10:47:00
2021-03-21 10:00:00


In [7]:
data = []
columns = ['date', 'title', 'href']

for post in posts:
   
    flag = False
    #поиск с учетом тэгов и превью статьи
    for keyword in KEYWORDS:
        flag = flag or (keyword in post.text)
            
    if flag:        
        title_element = post.find('a', class_='post__title_link')
        date_text = post.find('span', class_='post__time').text
        
        date_correct = get_correct_datetime(date_text)
        
        data.append([
                     date_correct,
                     title_element.text, 
                     title_element.attrs.get('href')
                    ])        

In [8]:
df = pd.DataFrame(data = data, columns = columns)
df

,date,title,href
0,2021-03-23 15:05:00,Обзор инструментов качества данных,https://habr.com/ru/post/548398/
1,2021-03-23 14:55:00,Что влияет на развитие отрасли видео-конференц...,https://habr.com/ru/company/ipmatika/blog/548546/
2,2021-03-23 13:09:00,"Эксперты о «среднем» диапазоне спектра от 3,3 ...",https://habr.com/ru/post/548524/
3,2021-03-23 12:35:00,Формульное определение проблем качества данных,https://habr.com/ru/post/548388/
4,2021-03-23 12:27:00,Подключённые автомобили: обобщённый метод взло...,https://habr.com/ru/company/trendmicro/blog/54...
5,2021-03-23 12:05:00,Русские хакерши: разговор с Алисой Esage Шевченко,https://habr.com/ru/company/ruvds/blog/548478/


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    6 non-null      datetime64[ns]
 1   title   6 non-null      object        
 2   href    6 non-null      object        
dtypes: datetime64[ns](1), object(2)
memory usage: 272.0+ bytes


**Задание 1. Дополнительная часть (необязательная)**

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [10]:
KEYWORDS = ['DQ (например, [13, 17])'] #https://habr.com/ru/post/548388/

In [12]:
data = []
columns = ['date', 'title', 'href', 'text']

for post in posts:
   
    title_element = post.find('a', class_='post__title_link')
    title = title_element.text
    href = title_element.attrs.get('href')
    
    date_text = post.find('span', class_='post__time').text
    date_correct = get_correct_datetime(date_text)
    
    #переходим по ссылке
    req_href = requests.get(href)
    soup_href = BeautifulSoup(req_href.text, 'lxml')
    post_href = soup_href.find('article', class_='post post_full')

    flag = False
    #поиск с учетом тэгов и полного текста статьи со страницы статьи
    for keyword in KEYWORDS:
        flag = flag or (keyword in post_href.text)
            
    if flag:        
        
        #текст статьи без залоговка и тэгов
        text = soup_href.find('div', class_='post__text post__text_v2').text
        
        data.append([
                     date_correct,
                     title, 
                     href,
                     text
                    ])        

In [13]:
df_add = pd.DataFrame(data = data, columns = columns)
df_add

,date,title,href,text
0,2021-03-23 12:35:00,Формульное определение проблем качества данных,https://habr.com/ru/post/548388/,A Formal Definition of Data Quality Problems (...


In [14]:
df_add.loc[0, 'text']

'A Formal Definition of Data Quality Problems (2005)\xa0Достаточно часто каждый аналитик сталкивается с ситуацией, когда загрузил данные в блок анализа, а в ответ – тишина, хотя в тестовом режиме все работает. Причина обычно в том, что данные недостаточно очищены, где в этой ситуации искать аналитику засаду и с чего начинать обычно задачка не из легких. Можно конечно использовать механизмы сглаживания, но каждый знает, что если из черного ящика с красными и зелеными шарами отсыпать килограмм шаров и вместо них вбросить килограмм белых, то в понимании распределения красных и зеленых это мало приблизит.Когда находишься в ситуации «а с чего начать» помогает таксономия «грязных данных». Хотя в учебниках и дают список проблем, но он обычно неполный, вот постоянно искал исследования, которые рассматривают эту тему подробней. Попалась работа T.Gschwandtner, J.Gartner, W.Aigner, S.Miksch хотя они ее делали для рассмотрения способов очистки данных связанных с датами и временем но, на мой взгляд

**Задание 2. Обязательная часть**

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Насk Сheck. Список email-ов задаем переменной в начале кода:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

In [15]:
EMAIL = ["aaa@list.ru", "bbb@list.ru"]
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [16]:
data_ = []
columns = ['email', 'date', 'source', 'description']

In [18]:
for mail in EMAIL:

    payload = {
        "emailAddresses" : [mail]
    } 

    headers = {
        "Vaar-Version": "0",
        "Vaar-Header-App-Product-Name": "hackcheck-web-avast",
        "Vaar-Header-App-Build-Version": "1.0.0"
    }
    res = requests.post(URL, data=json.dumps(payload), headers=headers)
    
    breaches = res.json()["breaches"]
    for key in breaches:
        
        data_.append([
                     mail,
                     datetime.strptime(breaches[key]['publishDate'], '%Y-%m-%dT%H:%M:%SZ'), #что это за Z - не нашла
                     breaches[key]['site'],
                     breaches[key]['description']
                    ])  

In [19]:
df_post = pd.DataFrame(data = data_, columns = columns)
df_post

,email,date,source,description
0,aaa@list.ru,2020-05-28,stalker.so,"In January 2020, the Russian multiplayer video..."
1,aaa@list.ru,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
2,aaa@list.ru,2020-07-23,wattpad.com,"In June 2020, the online writing community Wat..."
3,aaa@list.ru,2020-12-24,ledger.com,"In June 2020, the hardware-based cryptocurrenc..."
4,aaa@list.ru,2020-11-05,vimeworld.ru,"In January 2018, the Russian Minecraft server ..."
5,aaa@list.ru,2019-05-23,livejournal.com,"In 2017, social network LiveJournal's database..."
6,bbb@list.ru,2016-11-01,qip.ru,"In 2011, Russian instant messaging service pro..."
7,bbb@list.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
8,bbb@list.ru,2021-01-07,joygames.me,"In December 2020, the online gaming website Jo..."
9,bbb@list.ru,2017-03-01,asterios.ru,"In June 2010, Asterios' user database was alle..."
